In [1]:
import requests
from lxml import etree
from tqdm import tqdm
import time
import random
import pandas as pd
import re

In [2]:
name_list, content_list, date_list, score_list, city_list = [], [], [], [], []
movie_name = ""

In [3]:
def get_city(url, i):
    time.sleep(round(random.uniform(2, 3), 2))
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
    cookies = {'cookie': 'bid=Ge7txCUP3v4; ll="108303"; _vwo_uuid_v2=DB48689393ACB497681C7C540C832B546|f3d53bcb0314c9a34c861e9c724fcdec; ap_v=0,6.0; dbcl2="159607750:sijMjNWV7ek"; ck=kgmP; push_doumail_num=0; push_noty_num=0; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1549433417%2C%22https%3A%2F%2Fmovie.douban.com%2Fsubject%2F26266893%2Fcomments%3Fsort%3Dnew_score%26status%3DP%22%5D; _pk_ses.100001.8cb4=*; __lnkrntdmcvrd=-1; __yadk_uid=KqejvPo3L0HIkc2Zx7UXOJF6Vt9PpoJU; _pk_id.100001.8cb4=91514e1ada30bfa5.1549433417.1.1549433694.1549433417'}  # 2018.7.25修改，
    res = requests.get(url, cookies=cookies, headers=headers)
    if (res.status_code == 200):
        print("\n成功获取第{}个用户城市信息！".format(i))
    else:
        print("\n第{}个用户城市信息获取失败".format(i))
    pattern = re.compile('<div class="user-info">.*?<a href=".*?">(.*?)</a>', re.S)
    item = re.findall(pattern, res.text)  # list类型
    return (item[0])  # 只有一个元素，所以直接返回

In [4]:
def get_content(id, page):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
    cookies = {'cookie': 'll="108296"; bid=syR5NswPT5o; ap_v=0,6.0; __utma=30149280.1398452742.1550195668.1550195668.1550195668.1; __utmc=30149280; __utmz=30149280.1550195668.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); _vwo_uuid_v2=DD937D5F15D956BE652C4CDDD418B7AF4|a4770d84dbf9310627f91765bd222aa1; _pk_id.100001.8cb4=efed61ac14f0a3f5.1550195981.1.1550195981.1550195981.; _pk_ses.100001.8cb4=*; __utmt=1; __utmb=30149280.1.10.1550195668'}
    url = "https://movie.douban.com/subject/" + str(id) + "/comments?start=" + str(page * 10) + "&limit=20&sort=new_score&status=P"
    res = requests.get(url, headers=headers, cookies=cookies)

    pattern = re.compile('<div id="wrapper">.*?<div id="content">.*?<h1>(.*?) 短评</h1>', re.S)
    global movie_name
    movie_name = re.findall(pattern, res.text)[0]  # list类型

    res.encoding = "utf-8"
    if (res.status_code == 200):
        print("\n第{}页短评爬取成功！".format(page + 1))
        print(url)
    else:
        print("\n第{}页爬取失败！".format(page + 1))
    with open('html.html', 'w', encoding='utf-8') as f:
        f.write(res.text)
        f.close()
    x = etree.HTML(res.text)
    for i in range(1, 21):   # 每页20个评论用户
        name = x.xpath('//*[@id="comments"]/div[{}]/div[2]/h3/span[2]/a/text()'.format(i))
        # 下面是个大bug，如果有的人没有评分，但是评论了，那么score解析出来是日期，而日期所在位置spen[3]为空
        score = x.xpath('//*[@id="comments"]/div[{}]/div[2]/h3/span[2]/span[2]/@title'.format(i))
        date = x.xpath('//*[@id="comments"]/div[{}]/div[2]/h3/span[2]/span[3]/@title'.format(i))
        m = '\d{4}-\d{2}-\d{2}'
        try:
            match = re.compile(m).match(score[0])
        except IndexError:
            break
        if match is not None:
            date = score
            score = ["null"]
        else:
            pass
        content = x.xpath('//*[@id="comments"]/div[{}]/div[2]/p/span/text()'.format(i))
        id = x.xpath('//*[@id="comments"]/div[{}]/div[2]/h3/span[2]/a/@href'.format(i))
        try:
            city = get_city(id[0], i)  # 调用评论用户的ID城市信息获取
        except IndexError:
            city = " "
        name_list.append(str(name[0]))
        score_list.append(str(score[0]).strip('[]\''))  # bug 有些人评论了文字，但是没有给出评分
        date_list.append(str(date[0]).strip('[\'').split(' ')[0])
        content_list.append(str(content[0]).strip())
        city_list.append(city)

In [5]:
def main(ID, pages):
    global movie_name
    for i in tqdm(range(0, pages)):  # 豆瓣只开放500条评论
        get_content(ID, i)  # 第一个参数是豆瓣电影对应的id序号，第二个参数是想爬取的评论页数
        time.sleep(round(random.uniform(3, 5), 2))
    infos = {'name': name_list, 'city': city_list, 'content': content_list, 'score': score_list, 'date': date_list}
    data = pd.DataFrame(infos, columns=['name', 'city', 'content', 'score', 'date'])
    data.to_csv(movie_name + ".csv")  # 存储名为  电影名.csv

In [6]:
main(26266893, 5)  # 评论电影的ID号+要爬取的评论页面数

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]


第1页短评爬取成功！
https://movie.douban.com/subject/26266893/comments?start=0&limit=20&sort=new_score&status=P

成功获取第1个用户城市信息！

成功获取第2个用户城市信息！

成功获取第3个用户城市信息！

成功获取第4个用户城市信息！

成功获取第5个用户城市信息！

成功获取第6个用户城市信息！

成功获取第7个用户城市信息！

成功获取第8个用户城市信息！

成功获取第9个用户城市信息！

成功获取第10个用户城市信息！

成功获取第11个用户城市信息！

成功获取第12个用户城市信息！

成功获取第13个用户城市信息！

成功获取第14个用户城市信息！

成功获取第15个用户城市信息！

成功获取第16个用户城市信息！

成功获取第17个用户城市信息！

成功获取第18个用户城市信息！

成功获取第19个用户城市信息！

成功获取第20个用户城市信息！


 20%|████████████████▊                                                                   | 1/5 [01:09<04:38, 69.74s/it]


第2页短评爬取成功！
https://movie.douban.com/subject/26266893/comments?start=10&limit=20&sort=new_score&status=P

成功获取第1个用户城市信息！

成功获取第2个用户城市信息！

成功获取第3个用户城市信息！

成功获取第4个用户城市信息！

成功获取第5个用户城市信息！

成功获取第6个用户城市信息！

成功获取第7个用户城市信息！

成功获取第8个用户城市信息！

成功获取第9个用户城市信息！

成功获取第10个用户城市信息！

成功获取第11个用户城市信息！

成功获取第12个用户城市信息！

成功获取第13个用户城市信息！

成功获取第14个用户城市信息！

成功获取第15个用户城市信息！

成功获取第16个用户城市信息！

成功获取第17个用户城市信息！

成功获取第18个用户城市信息！

成功获取第19个用户城市信息！

成功获取第20个用户城市信息！


 40%|█████████████████████████████████▌                                                  | 2/5 [02:17<03:27, 69.22s/it]

...

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [03:28<02:19, 69.72s/it]


第4页短评爬取成功！
https://movie.douban.com/subject/26266893/comments?start=30&limit=20&sort=new_score&status=P

成功获取第1个用户城市信息！

成功获取第2个用户城市信息！

成功获取第3个用户城市信息！

成功获取第4个用户城市信息！

成功获取第5个用户城市信息！

成功获取第6个用户城市信息！

成功获取第7个用户城市信息！

成功获取第8个用户城市信息！

成功获取第9个用户城市信息！

成功获取第10个用户城市信息！

成功获取第11个用户城市信息！

成功获取第12个用户城市信息！

成功获取第13个用户城市信息！

成功获取第14个用户城市信息！

成功获取第15个用户城市信息！

成功获取第16个用户城市信息！

成功获取第17个用户城市信息！

成功获取第18个用户城市信息！

成功获取第19个用户城市信息！

成功获取第20个用户城市信息！


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [04:43<01:11, 71.12s/it]


第5页短评爬取成功！
https://movie.douban.com/subject/26266893/comments?start=40&limit=20&sort=new_score&status=P

成功获取第1个用户城市信息！

成功获取第2个用户城市信息！

成功获取第3个用户城市信息！

成功获取第4个用户城市信息！

成功获取第5个用户城市信息！

成功获取第6个用户城市信息！

成功获取第7个用户城市信息！

成功获取第8个用户城市信息！

成功获取第9个用户城市信息！

成功获取第10个用户城市信息！

成功获取第11个用户城市信息！

成功获取第12个用户城市信息！

成功获取第13个用户城市信息！

成功获取第14个用户城市信息！

成功获取第15个用户城市信息！

成功获取第16个用户城市信息！

成功获取第17个用户城市信息！

成功获取第18个用户城市信息！

成功获取第19个用户城市信息！

成功获取第20个用户城市信息！


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [05:58<00:00, 72.54s/it]
